- Explicit-Explicit Discovery: predict Arg2 sent-initial connective
- Explicit-Explicit PDTB: predict subsequent connective
- Explicit-Implicit: predict implicit 

In [85]:
import openai
import pandas as pd
import os
from tqdm import tqdm
from IPython.display import display, HTML


In [46]:
introduce_marker_symbol = '''\n  - Take note of the existing discourse marker enclosed by the character [ and ]'''

def prepare_system_prompt(type):
    prompt_file = ""
    if type == "free-insert":
        prompt_file = "free-insert_system_prompt.txt"
    elif type == "free-insert-co-occur":
        prompt_file = "free-insert-coo_system_prompt.txt"

    elif type == "mask-fill" or type == "mask-fill-with-marker-symbol":
        prompt_file = "mask-fill_system_prompt.txt"

    res = ""
    with open(prompt_file) as f:
        res = f.read()
    if "with-marker-symbol" in type:
        res = res.replace("<introduce_marker_symbol>", introduce_marker_symbol).replace("are curly and ___ are more", "are curly [and] ___ are more")
    else:
        res = res.replace("<introduce_marker_symbol>", "")
    return res

def prepare_user_prompt(arg1, conn1, arg2, type):
    res = ""
    with open("user_prompt.txt") as f:
        res = f.read()

    should_capitalize = arg1.rstrip().endswith(('.', '!', '?'))
    conn1 = conn1.capitalize() if should_capitalize else conn1

    if type == "free-insert" or type == "free-insert-co-occur":
        res = res.replace("<passage>", arg1 + " [" + conn1 + "] " + arg2)
    elif type == "mask-fill-with-marker-symbol":
        res = res.replace("<passage>", arg1 + " [" + conn1 + "] ___ " + arg2)
    elif type == "mask-fill":
        res = res.replace("<passage>", arg1 + " " + conn1 + " ___ " + arg2)
    return res


In [ ]:
arg1 = '''Jacobs may have gotten a temporary reprieve in the eyes of many rank-and-file fans with the hiring of Coach Malzahn, who has made all the right moves so far.'''
conn1 = "ultimately,"
arg2 = '''jacobs is the problem - or, at least, the public face of the problem.'''

In [ ]:
for type in ["mask-fill", "mask-fill-with-marker-symbol", "free-insert", "free-insert-co-occur"]:
    
    print(f"*************** {type} ***************")
    system_prompt = prepare_system_prompt(type)
    user_prompt = prepare_user_prompt(arg1, conn1, arg2, type)

    # with open(f"samples/{type}.txt", "w") as f:
    #     f.write(system_prompt + "\n\n" + user_prompt)
    print(system_prompt + "\n\n" + user_prompt)
    print("\n")

In [ ]:
client = openai.OpenAI(
    api_key="",

)
def get_response (arg1, conn1, arg2, type):
    response = client.chat.completions.create(
        model="gpt-4.1-2025-04-14", 
        messages = [
            {
                "role": "developer",
                "content": prepare_system_prompt(type)
            },
            {
                "role": "user",
                "content": prepare_user_prompt(arg1, conn1, arg2, type)
            }
        ]
    )
    return response.choices[0].message.content

In [174]:
folder = "../../dataset"
os.chdir(folder)

In [175]:
test_set = pd.read_csv("../dataset/explicit-explicit/discovery/dm1_pdtb/test.csv")
test_set

,id,sentence1,sentence2,dm1,dm2
0,1502,You don't know if it's true or not as a matter...,no one should judge and say anything and let t...,however,therefore
1,2992,Bloomberg's sensitivity to the climate issue w...,climate change had the last laugh at our polit...,"in the end,","however,"
2,1588,If you recall this infomercial from December 2...,it's been a tough year for tv's pitchmen.,"overall,","though,"
3,2363,"If having specific fonts and layouts matter, P...",it's what you say and not how you say it.,"ultimately,","though,"
4,6241,Onclick attribute of an anchor that goes to th...,"return false, to avoid any confusion.","anyway,",and
...,...,...,...,...,...
253,1798,A device to illustrate silver can be considere...,i wouldn't want to start such a conversation r...,"although,","clearly,"
254,6206,Then I give em 60 days during which I'm not go...,i've found you've got to play a little hardball.,but,"after that,"
255,1430,About the bus ride there.,"this is the worst for me, realizing that somet...","or,",and
256,455,It has been suggested Horncastle was a storage...,why the need for such strong walls?,"again,","though,"


In [179]:
!pwd

/Users/davidliu/co-occur_dm/dataset


In [182]:
folder = "../model/prompt"
os.chdir(folder)

# Cloze

In [183]:
def get_dm_for_test_set (df, type):
    result = []
    for i, r in tqdm(df.iterrows(), total=df.shape[0]):
        sentence1, sentence2, dm1 = r["sentence1"], r["sentence2"], r["dm1"]
        response = get_response(arg1, arg2, dm1, type)
        response = response.split(", ")
        result.append(response)
    return result

def format_dm(xss):
    return [[x.lower().strip() for x in xs] for xs in xss]

In [184]:
mask_fill = get_dm_for_test_set(test_set, 
                                "mask-fill")

100%|██████████| 258/258 [03:03<00:00,  1.41it/s]


In [185]:
mask_fill = format_dm(mask_fill)
test_set["cloze"] = mask_fill

In [186]:
len(test_set[test_set.apply(lambda r: r["dm2"].lower().rstrip(", ") in r["cloze"], axis=1)])/ len(test_set)

0.32558139534883723

# Cloze DM marked

In [187]:
cloze_dm_marked = get_dm_for_test_set(test_set, 
                                "mask-fill-with-marker-symbol")

100%|██████████| 258/258 [03:30<00:00,  1.22it/s]


In [188]:
cloze_dm_marked = format_dm(cloze_dm_marked)

In [189]:
test_set["cloze_dm_marked"] = cloze_dm_marked

In [157]:
test_set[test_set.apply(lambda r: r["dm2"].lower().rstrip(", ") not in r["cloze_dm_marked"], axis=1)]

,id,sentence1,sentence2,dm1,dm2,cloze,cloze_dm_marked
1,250,A victory?,i'm interested to see what the next couple of ...,"maybe,",but,"[however, still, nevertheless, yet, instead]","[however, still, yet, nevertheless, alternativ..."
2,3571,There does appear to be some real intelligence...,you can sort of separate some of the controver...,"secondly,","though,","[firstly, moreover, in addition, furthermore, ...","[firstly, moreover, in addition, furthermore, ..."
3,199,"Is it a call to action by Powell, wanting peop...","whatever the case, it's powerful stuff.","perhaps,",but,"[indeed, nevertheless, however, admittedly, ar...","[however, alternatively, on the other hand, ne..."
5,686,"That debate is constructive, as is Ben-Tor's c...",anti-ideology becomes an ideology itself.,"inevitably,","though,","[however, still, yet, nevertheless, ultimately]","[nevertheless, still, however, yet, ultimately]"
6,4822,Our parsha deals with all three roles.,the most attention-catching is the section on ...,"undoubtedly,","though,","[however, nevertheless, still, yet, indeed]","[however, nevertheless, still, yet, admittedly]"
...,...,...,...,...,...,...,...
926,4341,"Even now when you think of before and now, you...",you began living your weekends in a cage in he...,"soon,","though,","[however, nevertheless, still, yet, ultimately]","[however, nonetheless, still, ultimately, yet]"
927,6326,At least last night they started the show at t...,when the game's over and you're risking cuttin...,"really,","though,","[in fact, actually, indeed, essentially, ultim...","[but, indeed, in fact, actually, truly]"
928,5705,Note that the eBook versions are typically ava...,the price is the same for both formats (save e...,"absolutely,",and,"[indeed, certainly, unquestionably, undeniably...","[however, certainly, indeed, clearly, undeniably]"
929,3100,"In the good old days, which weren't actually a...",the chef shortage has turned his recruiting pr...,"lately,","though,","[however, nonetheless, still, yet, especially]","[however, nevertheless, on the other hand, sti..."


In [190]:
len(test_set[test_set.apply(lambda r: r["dm2"].lower().rstrip(", ") in r["cloze_dm_marked"], axis=1)])/len(test_set)

0.34108527131782945

# Free Insert

In [191]:
free_insert = get_dm_for_test_set(test_set, 
                                "free-insert")

100%|██████████| 258/258 [02:56<00:00,  1.46it/s]


In [192]:
free_insert = format_dm(free_insert)
test_set["free_insert"] = free_insert

In [194]:
len(test_set[test_set.apply(lambda r: r["dm2"].lower().rstrip(", ") in r["free_insert"], axis=1)])/len(test_set)

0.34108527131782945

# Free-Insert-COO

In [91]:
free_insert_coo = get_dm_for_test_set(test_set, 
                                "free-insert-co-occur")

100%|██████████| 203/203 [02:19<00:00,  1.45it/s]


In [92]:
free_insert_coo = format_dm(free_insert_coo)
test_set["free_insert_coo"] = free_insert_coo

In [93]:
len(test_set[test_set.apply(lambda r: r["dm2"].lower().rstrip(", ") in r["free_insert_coo"], axis=1)])/len(test_set)

0.2857142857142857

In [197]:
test_set.to_pickle("gpt4.1_dm1_pdtb_result.pkl")

In [172]:
df = pd.read_pickle("/Users/davidliu/co-occur_dm/model/prompt/result/discovery_dm1_other/gpt4.1_dm1_other_result.pkl")
df

,id,sentence1,sentence2,dm1,dm2,cloze,cloze_dm_marked,free_insert
0,3282,", and proudly continues that tradition of care...",i have been trying to drag myself out of this ...,"lately,","however,","[however, still, yet, nonetheless, recently]","[however, nevertheless, still, in fact, actually]","[however, consequently, in fact, moreover, thus]"
1,250,A victory?,i'm interested to see what the next couple of ...,"maybe,",but,"[however, still, nevertheless, yet, instead]","[however, still, yet, nevertheless, alternativ...","[however, thus, in fact, conversely, neverthel..."
2,3571,There does appear to be some real intelligence...,you can sort of separate some of the controver...,"secondly,","though,","[firstly, moreover, in addition, furthermore, ...","[firstly, moreover, in addition, furthermore, ...","[however, consequently, furthermore, moreover,..."
3,199,"Is it a call to action by Powell, wanting peop...","whatever the case, it's powerful stuff.","perhaps,",but,"[indeed, nevertheless, however, admittedly, ar...","[however, alternatively, on the other hand, ne...","[however, moreover, in fact, additionally, non..."
4,1935,She then accepted employment in one of the mor...,she did not feel that she deserved this state ...,"apparently,","however,","[however, nonetheless, still, yet, nevertheless]","[nonetheless, however, still, yet, nevertheless]","[however, nonetheless, still, thus, moreover]"
...,...,...,...,...,...,...,...,...
927,6326,At least last night they started the show at t...,when the game's over and you're risking cuttin...,"really,","though,","[in fact, actually, indeed, essentially, ultim...","[but, indeed, in fact, actually, truly]","[however, indeed, furthermore, nevertheless, b..."
928,5705,Note that the eBook versions are typically ava...,the price is the same for both formats (save e...,"absolutely,",and,"[indeed, certainly, unquestionably, undeniably...","[however, certainly, indeed, clearly, undeniably]","[indeed, in fact, clearly, namely, specifically]"
929,3100,"In the good old days, which weren't actually a...",the chef shortage has turned his recruiting pr...,"lately,","though,","[however, nonetheless, still, yet, especially]","[however, nevertheless, on the other hand, sti...","[however, nevertheless, still, consequently, d..."
930,296,But will that be Djokovic?,a djokovic in what condition?,"probably,",but,"[indeed, in fact, most likely, consequently, t...","[thus, in fact, indeed, consequently, still]","[however, nevertheless, still, in fact, conver..."


In [195]:
HTML(test_set.to_html())

,id,sentence1,sentence2,dm1,dm2,cloze,cloze_dm_marked,free_insert
0,1502,"You don't know if it's true or not as a matter of fact, no one knows for sure but the parties involved and God!",no one should judge and say anything and let the legal counsels do what they must do.,however,therefore,"[still, nonetheless, nevertheless, even so, that said]","[nevertheless, still, nonetheless, conversely, yet]","[nonetheless, still, conversely, on the other hand, yet]"
1,2992,"Bloomberg's sensitivity to the climate issue was heightened, perhaps, by what Sandy did to his city.",climate change had the last laugh at our political process.,"in the end,","however,","[nevertheless, ultimately, however, still, yet]","[still, nevertheless, however, yet, ultimately]","[however, moreover, still, consequently, ultimately]"
2,1588,"If you recall this infomercial from December 2008, you probably remember it for lame double entendres like ""You're gonna love my nuts.""",it's been a tough year for tv's pitchmen.,"overall,","though,","[however, still, nevertheless, yet, but]","[nonetheless, still, however, nevertheless, yet]","[however, nevertheless, in fact, similarly, consequently]"
3,2363,"If having specific fonts and layouts matter, PDF is the way to go.",it's what you say and not how you say it.,"ultimately,","though,","[but, yet, still, nevertheless, however]","[however, still, nevertheless, yet, but]","[however, nevertheless, in fact, furthermore, instead]"
4,6241,Onclick attribute of an anchor that goes to the page.,"return false, to avoid any confusion.","anyway,",and,"[still, however, nevertheless, regardless, yet]","[still, nevertheless, regardless, however, ultimately]","[however, nevertheless, still, in fact, thus]"
5,939,Where's the invention?,cingular's patent application is much broader than a simple emoticon key.,"indeed,",and,"[in fact, actually, certainly, undoubtedly, truly]","[however, in fact, actually, moreover, certainly]","[however, nonetheless, furthermore, nevertheless, moreover]"
6,6193,"They figure the banks will eventually lend out these $1.8 trillion of reserves, and this will make prices explode.",they think the fed will soon have to start hiking interest rates (including interest on reserves) to head off such a stagflationary nightmare.,"or,",alternatively,"[alternatively, conversely, in other words, rather, instead]","[however, nevertheless, still, yet, alternatively]","[however, nevertheless, still, instead, yet]"
7,3609,Harry didn't think so either.,"the gryffindor hadn't really thought of sex, or having a lover, just surviving another year.","though,","in fact,","[however, nevertheless, still, even so, yet]","[however, still, nevertheless, even so, yet]","[however, nonetheless, still, yet, ultimately]"
8,1540,"The artist was engaged by the publisher to make the sketch and having been compensated, to a large extent had no further interest in the process.",the cutter and then the printer.,"likewise,","in turn,","[similarly, furthermore, in addition, moreover, also]","[similarly, indeed, furthermore, in fact, moreover]","[however, in contrast, still, moreover, additionally]"
9,4965,"While many readers wouldn't be bothered, I was.","i truly enjoyed chaos burning, and would continue to read ms. dane's titles in her bound by magick series.","ultimately,","however,","[however, nevertheless, still, yet, thus]","[however, still, nevertheless, yet, but]","[however, furthermore, nevertheless, consequently, still]"
